In [152]:
from MCpredict import *
import pandas as pd
import ipywidgets as widgets
from collections import defaultdict
import collections
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import HBox, VBox

************************************ EXAMPLE ************************************************

In [153]:
pred_3 = MCpredict(3)
pred_4 = MCpredict(4)

In [154]:
print(pred_3.predict_seq('Wood,Trsgi'))
out_list = pred_4.predict_seq('MarineSediment')
print(out_list)

{'0': ['Wood'], '1': ['Trsgi'], '2': ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite']}
{'0': ['MarineSediment'], '1': ['D18O', 'Depth', 'Mg/Ca', 'Uk37Prime', 'D13C'], '2': [['T', 'Temperature', 'NA', 'D18Osw', 'Temperature And Salinity'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Temperature', 'NA', 'Wood', 'MXD', 'Composite'], ['Temperature', 'NA', 'Wood', 'MXD', 'Composite'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite']], '3': [[['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', 'Wood'], ['Sea Surface', 'Thermocline', 'Bottom Water', 'Surface Water', 'Subsurface'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Sea Surface', 'Bottom Water', 'Surface Water', 'Surface', 'Subsurface'], ['Sea Surface', 'Wood', 'MXD', 'LakeSediment', 'Composite']], [['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core'], ['T', 'NA', 'Wood', 'MXD', 'Composite'], ['Thickness', 'Depth', 'Core', 'Uk37', 'TEX86'], ['T', 'Wood', 'MXD', 'La

## DEMO ##

In [196]:
chain_length_dropdown = widgets.Dropdown(options = [3,4], description='Chain Type')
predict_for_chain_type = MCpredict(3)
input_sent = ''

In [197]:
archive_type_dropdown = widgets.Dropdown(options = sorted(predict_for_chain_type.archive_types), description='Archive Type')

In [198]:
output_proxy = widgets.Output()
output_units = widgets.Output()
output_int_var = widgets.Output()
output_int_var_det = widgets.Output()

In [199]:
style = {'description_width': 'initial', 'select_width' : 'initial'}

# proxy_obs_type_dropdown = widgets.Combobox(options = [], description='Proxy Observation Type', style=style, ensure_option=True,
#     disabled=False)
# units_dropdown = widgets.Combobox(options = [], description='Units', style=style, ensure_option=True,
#     disabled=False)
# int_var_dropdown = widgets.Combobox(options = [], description='Interpretation Variable', style=style, ensure_option=True,
#     disabled=False)
# int_var_det_dropdown = widgets.Combobox(options = [], description='Interpretation Variable Detail', style=style, ensure_option=True,
#     disabled=False)

proxy_obs_type_dropdown = widgets.Dropdown(options = [], description='Proxy Observation Type', style=style)
units_dropdown = widgets.Dropdown(options = [], description='Units', style=style)
int_var_dropdown = widgets.Dropdown(options = [], description='Interpretation Variable', style=style)
int_var_det_dropdown = widgets.Dropdown(options = [], description='Interpretation Variable Detail', style=style, ensure_option=True,
    disabled=False)

In [200]:
layout_hidden  = widgets.Layout(visibility = 'hidden')
layout_visible = widgets.Layout(visibility = 'visible')

In [201]:
def visible_txt(b):
    combobox.layout = layout_visible

def hidden_txt(b):
    combobox.layout = layout_hidden

In [202]:
def displayChildren(options_list, dropdown_obj, dropdown_eventhandler, to_observe):
#     print('inside display children')
#     print(options_list)
    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')

In [203]:
def int_var_dropdown_eventhandler(change):
    global input_sent
    
    output_int_var_det.clear_output()
#     int_var_det_dropdown.layout = layout_visible
    
    int_var_value = change.new
    input_sent += ',' + int_var_value
    results = predict_for_chain_type.predict_seq(input_sent)

    displayChildren(results['3'], int_var_det_dropdown, None, False)
    with output_int_var_det:
        display(int_var_det_dropdown)

In [204]:
def proxy_obs_type_dropdown_eventhandler(change):
    global input_sent
    
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    
    proxy_obs_type_value = change.new
    input_sent += ',' + proxy_obs_type_value
    results = predict_for_chain_type.predict_seq(input_sent)
    
    if predict_for_chain_type.chain_length == 3:
#         units_dropdown.layout = layout_visible
        displayChildren(results['2'], units_dropdown, None, False)
        with output_units:
            display(units_dropdown)
    else:
#         int_var_dropdown.layout = layout_visible
        displayChildren(results['2'], int_var_dropdown, int_var_dropdown_eventhandler, True)
        with output_int_var:
            display(int_var_dropdown)

In [205]:
def archive_type_dropdown_eventhandler(change):
    global input_sent
    
    # value from archive type dropdown
    archive_type_value = change.new
    input_sent = archive_type_value
    
    # clear existing values from all children drop downs
    output_proxy.clear_output()
    
    output_units.clear_output()
#     units_dropdown.layout = layout_hidden
    
    output_int_var.clear_output()
    output_int_var_det.clear_output()
#     int_var_dropdown.layout = layout_hidden
#     int_var_det_dropdown.layout = layout_hidden
    
    # get prediction results from MC model
    results = predict_for_chain_type.predict_seq(input_sent)
    print('results', results)
    displayChildren(results['1'], proxy_obs_type_dropdown, proxy_obs_type_dropdown_eventhandler, True)
    
    with output_proxy:
        display(proxy_obs_type_dropdown)
    
# archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

# display(archive_type_dropdown)

In [206]:
def chain_type_dropdown_eventhandler(change):
    global input_sent, predict_for_chain_type
    
    predict_for_chain_type = MCpredict(change.new)
    
    input_sent = ''
    
    output_proxy.clear_output()
    
    output_units.clear_output()
#     units_dropdown.layout = layout_hidden
    
    output_int_var.clear_output()
    output_int_var_det.clear_output()
#     int_var_dropdown.layout = layout_hidden
#     int_var_det_dropdown.layout = layout_hidden
    
    archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')
    display(archive_type_dropdown)

In [207]:
chain_length_dropdown.observe(chain_type_dropdown_eventhandler, names='value')

display(chain_length_dropdown)

Dropdown(description='Chain Type', options=(3, 4), value=3)

Dropdown(description='Archive Type', options=('Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'MarineSediment…

results {'0': ['GlacierIce'], '1': ['D18O', 'Dd', 'Thickness', 'Depth', 'Wood'], '2': [['T', 'Temperature', 'NA', 'D18Osw', 'Temperature And Salinity'], ['T', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['T', 'NA', 'Wood', 'MXD', 'Composite'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core']], '3': [[['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', 'Wood'], ['Sea Surface', 'Thermocline', 'Bottom Water', 'Surface Water', 'Subsurface'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['Sea Surface', 'Bottom Water', 'Surface Water', 'Surface', 'Subsurface'], ['Sea Surface', 'Wood', 'MXD', 'LakeSediment', 'Composite']], [['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', 'Wood'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core'], ['T', 'NA', 'Wood', 'MXD', 'Composite'], ['Thickness', 'Depth', 'Core', 'Uk37', 'TEX86'], ['T', 'Wood', 'MXD', 'LakeSediment', 'Composite']], [['Sea Surface', 'Air Surface', 'Air Condensationlevel', 'NA', '

Dropdown(description='Archive Type', index=1, options=('Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'Marin…

results {'0': ['Coral'], '1': ['D18O', 'Calcification', 'Sr/Ca', 'D13C', 'Composite'], '2': [['permil', 'per mil', 'per mil VPDB', 'per mil PDB', 'per mil SMOW'], ['g/cm/yr', 'mm/yr', 'cm/yr', 'Wood', 'MXD'], ['mmol/mol', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['permil', 'per mil', 'Wood', 'MXD', 'Composite'], ['percent', 'Wood', 'MXD', 'LakeSediment', 'Composite']]}
results {'0': ['Speleothem'], '1': ['D18O', 'SE', 'Wood', 'MXD', 'Composite'], '2': [['permil', 'per mil', 'per mil VPDB', 'per mil PDB', 'per mil SMOW'], ['NA', 'degC', 'Wood', 'MXD', 'Composite'], ['Trsgi', 'ARS', 'EPS', 'SD', 'Core'], ['NA', 'Wood', 'MXD', 'LakeSediment', 'Composite'], ['percent', 'Wood', 'MXD', 'LakeSediment', 'Composite']]}


In [195]:
if predict_for_chain_type.chain_length == 3:
    first_line = output_proxy
    second_line = output_units
    VBox([first_line, second_line])
else:
    first_line = output_proxy
    second_line = output_int_var
    third_line = output_int_var_det
    VBox([first_line, second_line, third_line])